In [1]:
import pandas as pd
from PIL import Image
import os
import numpy as np
from xgboost import XGBRegressor
from xgboost import XGBClassifier, plot_importance
import torch,torchvision
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.decomposition import PCA
from tqdm import tqdm
import seaborn as sns 
import matplotlib.pyplot as plt
import random
import re
import time



from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.linear_model import TheilSenRegressor
from sklearn.tree import ExtraTreeRegressor


In [ ]:
!python --version

In [2]:
df = pd.read_csv('./h5bench-small-hue-final.csv') #

In [7]:

x = df[['imgFilename','stripeCount', 'stripeSize', 'runTime', 'fileSize', 'nodeNum', 'procNum']]
y = df[['Performance']]

In [8]:
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.2,random_state=7)

In [9]:
x_train.shape

(120, 7)

In [10]:
preprocess = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [11]:
## yes image but no runtime
def getFeatures(x_df,model):

    train_fc = []
    
    for _,val in tqdm(x_df.iterrows()):
        img = Image.open(val['imgFilename']).convert('RGB')
        input_tensor = preprocess(img)
        out_fc = model(input_tensor.unsqueeze(0)).squeeze(0).numpy()

        combinedfc = np.append(out_fc,val['stripeCount'])
        combinedfc = np.append(combinedfc,val['stripeSize'])
        combinedfc = np.append(combinedfc,val['fileSize'])
        combinedfc = np.append(combinedfc,val['nodeNum'])
        combinedfc = np.append(combinedfc,val['procNum'])
        train_fc.append(combinedfc)
    train_fc = np.stack(train_fc)
    return train_fc

In [12]:
model = torchvision.models.vgg16(weights=True)
model = model.eval()
for param in model.parameters():
    param.requires_grad = False

/global/homes/s/sgkim/.local/perlmutter/python-3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-7])

In [14]:
features = getFeatures(x_train,model)

120it [00:15,  7.60it/s]


In [15]:
testfeatures = getFeatures(x_test, model)

30it [00:03,  8.03it/s]


In [16]:
print(features.shape)
print(testfeatures.shape)

(120, 25093)
(30, 25093)


In [18]:
# reg = XGBRegressor()
# reg = RandomForestRegressor() 
##reg = ExtraTreesRegressor(n_estimators=5, max_depth=10, max_features=0.3, n_jobs=-1, random_state=0) 
reg = ExtraTreesRegressor() # 

In [19]:
start = time.time()

reg.fit(features,y_train)

print(f'Time: {time.time() - start}')

/global/common/software/nersc/pe/conda-envs/23.10.0/python-3.11/nersc-python/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Time: 7.618940591812134


In [15]:
score = reg.score(features, y_train)  
print("Training score: ", score)

Training score:  1.0


In [16]:
pred = reg.predict(testfeatures)

In [17]:
print(pred.shape)
print(y_test.shape)

(30,)
(30, 1)


In [18]:
r2score = r2_score(y_test, pred)
print("R2 score: ", r2score)

R2 score:  0.9360856842367715


In [ ]:
sns.regplot(x = y_test,y = pred);

plt.title('Regression Plot on Tested Data', fontsize=18)
plt.xlabel('Actual Performance', fontsize=18)
plt.ylabel('Predicted Performance',fontsize=18)
plt.gcf().set_dpi(600)
plt.show()

In [ ]:
######################################################################

In [ ]:
# imgFilename	stripeCount	stripeSize	runTime	Performance	fileSize	nodeNum	procNum	dimNum
r2Sum = 0
cnt = 0
maxR2score = 0
maxImg = ""

for idx, row in tqdm(x_test.iterrows()):
    df_row = pd.DataFrame(data=[[row['imgFilename'], row['stripeCount'], row['stripeSize'], row['runTime'], row['fileSize'], row['nodeNum'], row['procNum']]],
                         columns=['imgFilename', 'stripeCount', 'stripeSize', 'runTime', 'fileSize', 'nodeNum', 'procNum'])


    start = time.time()
    features_to_test_PFS = getFeatures_for_test_all_PFSconfig(df_row, model)
    print(f'Feature Time: {time.time() - start}')

    print(features_to_test_PFS)


    imgFilenameTest = row['imgFilename']
    print("Img Name is {}".format(imgFilenameTest))
    
    
    splitLine = imgFilenameTest.split("/")[2].split("-")

    jobID = re.findall(r'\d+',splitLine[4])[0]
    df_row_match = df[(df['imgFilename'].str.contains(splitLine[0]+"-"+splitLine[1]+"-"+splitLine[2]))]
    df_row_match = df_row_match.sort_values(by="stripeCount", ascending=True)

    perfList = list(df_row_match['Performance'])
    print("Actual Performance is {}".format(perfList))
    
    start = time.time()
    predTestPFS = reg.predict(features_to_test_PFS)
    print(f'Predict Time: {time.time() - start}')

    if len(perfList) == len(predTestPFS):
        r2score = r2_score(perfList, predTestPFS)
        r2Sum = r2Sum + r2score
        cnt = cnt + 1
        print("Predict Performance is {}".format(predTestPFS))
        print("r2 score is {}\n\n".format(r2score))
        

        xgraph = ["1", "8", "16", "32", "64"]
        sns.lineplot(x=xgraph,y=predTestPFS, color='red', marker='^', legend="auto");
        sns.lineplot(x=xgraph,y=perfList, color='blue', marker='o', legend="auto");
        plt.xlabel('stripe counts', fontsize=18)
        plt.ylabel('Reg Predicted Performance',fontsize=18)
        plt.legend(labels=["Predicted","Actual"])
        plt.gcf().set_dpi(600)
        plt.show()


        plt.show()
        
        if r2score >= maxR2score:
            maxR2score = r2score
            maxImg = imgFilenameTest
    else:
        print("No data match between pred and actual: skipping \n\n")
    # break


    
print("Avg R2score is {}".format(r2Sum / cnt))
print("Max R2score is {} / img: {}".format(maxR2score, maxImg))
    
    
